# [Build an Agent](https://python.langchain.com/docs/tutorials/agents/)

エージェントは、LLM を推論エンジンとして使用して、実行するアクションと、アクションを実行するために必要な入力を決定するシステムです。  
アクションを実行した後、結果を LLM にフィードバックして、さらにアクションが必要かどうか、または終了しても問題ないかを判断できます。

## LangSmith

https://smith.langchain.com/

## Tavily

https://tavily.com/


## リファレンス

- [TavilySearchResults](https://python.langchain.com/api_reference/community/tools/langchain_community.tools.tavily_search.tool.TavilySearchResults.html)


# Tavilyの定義

In [19]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in Tokyo")
print(search_results)

tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'Tokyo', 'region': 'Tokyo', 'country': 'Japan', 'lat': 35.6895, 'lon': 139.6917, 'tz_id': 'Asia/Tokyo', 'localtime_epoch': 1730643303, 'localtime': '2024-11-03 23:15'}, 'current': {'last_updated_epoch': 1730643300, 'last_updated': '2024-11-03 23:15', 'temp_c': 18.1, 'temp_f': 64.6, 'is_day': 0, 'condition': {'text': 'Clear', 'icon': '//cdn.weatherapi.com/weather/64x64/night/113.png', 'code': 1000}, 'wind_mph': 6.9, 'wind_kph': 11.2, 'wind_degree': 115, 'wind_dir': 'ESE', 'pressure_mb': 1022.0, 'pressure_in': 30.18, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 68, 'cloud': 0, 'feelslike_c': 18.1, 'feelslike_f': 64.6, 'windchill_c': 17.6, 'windchill_f': 63.7, 'heatindex_c': 17.6, 'heatindex_f': 63.7, 'dewpoint_c': 8.6, 'dewpoint_f': 47.5, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.0, 'gust_mph': 9.4, 'gust_kph': 15.2}}"}, {'url': 'https://weatherspark.com/h/m/143809/2024/3/Historical-Weather-in-March-2024-in-Toky

# エージェントを利用した推論

In [34]:
from langchain_openai import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.runnables import RunnableConfig

# モデル
model = ChatOpenAI(model="gpt-4o")

# メモリ
memory = MemorySaver()

# エージェント作成
search = TavilySearchResults(max_results=2)
tools = [search]
agent_executor = create_react_agent(model, tools, checkpointer=memory)

In [35]:
config = RunnableConfig(configurable={"thread_id": "abc123"})

# 自己紹介をしているだけなので、Tavilyにアクセスする必要はない
output_stream = agent_executor.stream(
    {"messages": [HumanMessage("hi im bob! and i live in sf")]},
    config
)

for chunk in output_stream:
    print(chunk)
    print("-----")

{'agent': {'messages': [AIMessage(content="Hi Bob! It's great to meet you. How's the weather in San Francisco today?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 88, 'total_tokens': 107, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_159d8341cc', 'finish_reason': 'stop', 'logprobs': None}, id='run-ac5a9768-d451-4160-adae-ecbaf663d7bd-0', usage_metadata={'input_tokens': 88, 'output_tokens': 19, 'total_tokens': 107, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 0}})]}}
-----


In [36]:
# MemorySaverに保存された情報から、住んでいる地域を取得して、Tavilyにアクセスして天気を取得する
output_stream = agent_executor.stream(
    {"messages": [HumanMessage("whats the weather where I live?")]},
    config
)

for chunk in output_stream:
    print(chunk)
    print("-----")


{'agent': {'messages': [AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Pa2EjEryBASO4j9kK7xma3SE', 'function': {'arguments': '{"query":"current weather San Francisco"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 122, 'total_tokens': 143, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_159d8341cc', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e69a7034-d05a-42ba-a988-a99af0396094-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather San Francisco'}, 'id': 'call_Pa2EjEryBASO4j9kK7xma3SE', 'type': 'tool_call'}], usage_metadata={'input_tokens': 122, 'output_tokens': 21, 'total_tokens': 143, 'input_token_details': {'cache_read': 0}, 'output_token_details'

# Not ストリーミング

In [38]:
config = RunnableConfig(configurable={"thread_id": "paenorbj"})
response = agent_executor.invoke(
    {"messages": [HumanMessage(content="What's the weather in Tokyo?")]},
    config,
)
response["messages"]

[HumanMessage(content="What's the weather in Tokyo?", additional_kwargs={}, response_metadata={}, id='ae4c1bfa-45d2-4369-a4b2-e40b8b220ff9'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Ymg3brnC4yjjK7VWG7u1Ntyb', 'function': {'arguments': '{"query":"current weather in Tokyo"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 85, 'total_tokens': 106, 'completion_tokens_details': {'audio_tokens': None, 'reasoning_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': None, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_159d8341cc', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f65c4945-12f1-40b8-a23f-a3ee41aa0d8d-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Tokyo'}, 'id': 'call_Ymg3brnC4yjjK7VWG7u1Ntyb', 'type': 'tool_call'}], usage_metadata={'input_tokens': 85,

# トークンストリーミング

In [40]:
config = RunnableConfig(configurable={"thread_id": "pa;oenb"})
async for event in agent_executor.astream_events(
    {"messages": [HumanMessage(content="whats the weather in sf?")]}, config, version="v1"
):
    kind = event["event"]
    if kind == "on_chain_start":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print(
                f"Starting agent: {event['name']} with input: {event['data'].get('input')}"
            )
    elif kind == "on_chain_end":
        if (
            event["name"] == "Agent"
        ):  # Was assigned when creating the agent with `.with_config({"run_name": "Agent"})`
            print()
            print("--")
            print(
                f"Done agent: {event['name']} with output: {event['data'].get('output')['output']}"
            )
    if kind == "on_chat_model_stream":
        content = event["data"]["chunk"].content
        if content:
            # Empty content in the context of OpenAI means
            # that the model is asking for a tool to be invoked.
            # So we only print non-empty content
            print(content, end="|")
    elif kind == "on_tool_start":
        print("--")
        print(
            f"Starting tool: {event['name']} with inputs: {event['data'].get('input')}"
        )
    elif kind == "on_tool_end":
        print(f"Done tool: {event['name']}")
        print(f"Tool output was: {event['data'].get('output')}")
        print("--")

--
Starting tool: tavily_search_results_json with inputs: {'query': 'San Francisco weather'}
Done tool: tavily_search_results_json
Tool output was: content='[{"url": "https://www.weatherapi.com/", "content": "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.775, \'lon\': -122.4183, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1730645131, \'localtime\': \'2024-11-03 06:45\'}, \'current\': {\'last_updated_epoch\': 1730645100, \'last_updated\': \'2024-11-03 06:45\', \'temp_c\': 12.8, \'temp_f\': 55.0, \'is_day\': 1, \'condition\': {\'text\': \'Partly cloudy\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/116.png\', \'code\': 1003}, \'wind_mph\': 9.8, \'wind_kph\': 15.8, \'wind_degree\': 323, \'wind_dir\': \'NW\', \'pressure_mb\': 1017.0, \'pressure_in\': 30.04, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 86, \'cloud\': 25, \'feelslike_c\': 11.3, \'feelslike_f\': 52.3, \'windchill_c\'